# 🔍 Análisis Exploratorio de Datos (EDA)

**Proyecto:** Detección de Fraude en Transacciones Financieras  
**Objetivo:** Realizar un análisis exhaustivo de los datos para comprender patrones, relaciones y características

---

## 📋 Índice del Análisis

1. **Carga de Datos y Configuración Inicial**
2. **Exploración Inicial de Datos**
3. **Caracterización de Variables**
4. **Limpieza y Preprocesamiento**
5. **Análisis Univariable**
6. **Análisis Bivariable**
7. **Análisis Multivariable**
8. **Identificación de Transformaciones**
9. **Reglas de Validación**
10. **Conclusiones y Siguientes Pasos**

# ==========================================
# 3. CARACTERIZACIÓN DE VARIABLES
# ==========================================

## 📝 Definición de las Variables

### **step** (Numérico, Entero)
- Representa una unidad de tiempo en el mundo real
- Cada paso (1 step) equivale a 1 hora
- El dataset cubre un total de 744 steps (30 días)

### **type** (Categórico, Nominal)
- Tipo de transacción financiera
- Valores posibles: CASH_IN, CASH_OUT, DEBIT, PAYMENT, TRANSFER

### **amount** (Numérico, Continuo)
- El monto de la transacción en la moneda local

### **nameOrig** (Categórico, Nominal)
- Identificador del cliente que inició la transacción
- Los clientes que inician con 'C' son clientes estándar

### **oldbalanceOrg** (Numérico, Continuo)
- Saldo del cliente de origen antes de la transacción

### **newbalanceOrig** (Numérico, Continuo)
- Saldo del cliente de origen después de la transacción

### **nameDest** (Categórico, Nominal)
- Identificador del cliente receptor (destino) de la transacción
- Los clientes que inician con 'M' son comerciantes (Merchants)

### **oldbalanceDest** (Numérico, Continuo)
- Saldo del cliente de destino antes de la transacción

### **newbalanceDest** (Numérico, Continuo)
- Saldo del cliente de destino después de la transacción

### **isFraud** (Binario, Target) 🎯
- **Variable objetivo (target)**
- Identifica si una transacción es fraudulenta (1) o legítima (0)
- El fraude ocurre exclusivamente en transacciones de tipo TRANSFER y CASH_OUT

### **isFlaggedFraud** (Binario, Flag)
- Bandera (flag) interna del sistema que marca transacciones sospechosas
- Se activa cuando se intenta transferir más de 200,000 en una sola transacción
- Es una regla de negocio simple, no es un resultado de un modelo

In [ ]:
# ==========================================
# 1. CARGA DE DATOS Y CONFIGURACIÓN INICIAL
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from scipy import stats
from scipy.stats import skew, kurtosis

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Configuración de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Librerías importadas correctamente")

In [ ]:
# Cargar los datos del notebook anterior
try:
    df = pd.read_pickle('../../data/processed/df_original.pkl')
    print("✅ Datos cargados desde archivo pickle")
except:
    df = pd.read_csv('../../Base_datos.csv')
    print("✅ Datos cargados desde CSV")

print(f"📊 Dimensiones del dataset: {df.shape[0]:,} filas x {df.shape[1]} columnas")
print(f"💾 Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

NameError: name 'df' is not defined

In [ ]:
# ==========================================
# 2. EXPLORACIÓN INICIAL DE DATOS
# ==========================================

print("=" * 80)
print("PRIMERAS Y ÚLTIMAS FILAS DEL DATASET")
print("=" * 80)
print("\n📋 Primeras 10 filas:")
display(df.head(10))

print("\n📋 Últimas 10 filas:")
display(df.tail(10))

In [ ]:
# Información general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
df.info()

print("\n📊 Tipos de datos:")
print(df.dtypes.value_counts())

In [ ]:
# Verificar valores nulos
print("=" * 80)
print("ANÁLISIS DE VALORES NULOS")
print("=" * 80)

valores_nulos = df.isnull().sum()
porcentaje_nulos = (df.isnull().sum() / len(df)) * 100

resumen_nulos = pd.DataFrame({
    'Valores_Nulos': valores_nulos,
    'Porcentaje': porcentaje_nulos
})

print(resumen_nulos)

if valores_nulos.sum() == 0:
    print("\n✅ EXCELENTE: No se encontraron valores nulos en el dataset")
else:
    print(f"\n⚠️ Se encontraron {valores_nulos.sum()} valores nulos")

In [ ]:
# Verificar duplicados
print("=" * 80)
print("ANÁLISIS DE DUPLICADOS")
print("=" * 80)

duplicados = df.duplicated().sum()
print(f"Número de filas duplicadas: {duplicados:,}")

if duplicados > 0:
    print(f"⚠️ Porcentaje de duplicados: {(duplicados/len(df))*100:.4f}%")
else:
    print("✅ No se encontraron filas duplicadas")

In [ ]:
# Caracterización detallada de variables
print("=" * 80)
print("CARACTERIZACIÓN DE VARIABLES")
print("=" * 80)

# Separar variables por tipo
variables_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
variables_categoricas = df.select_dtypes(include=['object']).columns.tolist()

print(f"\n📊 Variables Numéricas ({len(variables_numericas)}):")
for var in variables_numericas:
    print(f"  • {var}")

print(f"\n📊 Variables Categóricas ({len(variables_categoricas)}):")
for var in variables_categoricas:
    print(f"  • {var}")

# Clasificación más detallada
print("\n" + "=" * 80)
print("CLASIFICACIÓN DETALLADA DE VARIABLES")
print("=" * 80)

clasificacion = {
    'Numéricas Continuas': ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'],
    'Numéricas Discretas': ['step'],
    'Categóricas Nominales': ['type', 'nameOrig', 'nameDest'],
    'Binarias (Dicotómicas)': ['isFraud', 'isFlaggedFraud']
}

for categoria, variables in clasificacion.items():
    print(f"\n{categoria}:")
    for var in variables:
        print(f"  • {var}")

# ==========================================
# 4. LIMPIEZA Y PREPROCESAMIENTO
# ==========================================